In [1]:
#Libraries
#Dataframe and arrays
import pandas as pd
import numpy as np

#Graphing/ Charting/ Visualisations
import matplotlib as plt

In [2]:
df = pd.read_excel('../data/online_fashion.xlsx')

In [3]:
df_raw = df

In [4]:
#Find & Remove Strange StockCodes
df_unique_stock_codes = df['StockCode'].unique()
df_unique_stock_codes = pd.DataFrame(df_unique_stock_codes)
df_unique_stock_codes['len'] = df_unique_stock_codes[0].astype(str).str.len()
df_unique_stock_codes.rename(columns={0:'StockCode'}, inplace=True)
df_unique_stock_codes.sort_values(by='len')
df_unique_stock_codes['len'] = df_unique_stock_codes['len'].astype(int)
df_stock_to_drop_stg = df_unique_stock_codes.loc[(df_unique_stock_codes['len'] >= 8) | (df_unique_stock_codes['len'] <= 4)].sort_values(by='len')
df_stock_to_drop_stg.reset_index(drop=True)
df_stock_to_drop = pd.concat([df_stock_to_drop_stg.iloc[0:10], df_stock_to_drop_stg.iloc[-6:], df_stock_to_drop_stg.iloc[-9:-8] ])
df_stock_to_drop

df = df[~(df['StockCode'].isin(df_stock_to_drop.StockCode) )]

df.shape

(538959, 8)

In [5]:
df['Month'] = df['InvoiceDate'].dt.month_name()

C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
#Identify if Return(1) or Sale(0)
def transtype(row): 
    """
    Input:
    Output:
    
    The purpose of this function is to .....
    """
    if row['Quantity'] <= 0 : 
        return 1
    else: 
        return 0

df['Return'] = df.apply(transtype, axis=1)
df.head()

C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,December,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,December,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0


In [7]:
df['Rev'] = (df.Quantity * df.UnitPrice)
df.head()

C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,December,0,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,December,0,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34


In [8]:
#Number of rows with no CustomerID
len(df[df['CustomerID'].isnull()]) #134,050

134050

In [9]:
#Number of items with no CustomerID
df[df['CustomerID'].isnull()].Quantity.sum() #264,390

264390

In [10]:
#Number of items sold with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 0) )].Quantity.sum() #47,3893

473893

In [11]:
#Number of items returned with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 1) )].Quantity.sum() #-209,503

-209503

In [12]:
#Value of items sold with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 0) )].Rev.sum() #1,510,366.41

1510366.41

In [13]:
#Value of items returned with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 1) )].Rev.sum() -4,320.85

(-4324.85, 320.85)

In [14]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
Month                  object
Return                  int64
Rev                   float64
dtype: object

In [15]:
#Number of items returned with no CustomerID in May
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'May') )].Quantity.sum() #13,172

-13172

In [16]:
#Vale of items returned with no CustomerID in May
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'May') )].Rev.sum() #-19.98

-19.98

In [17]:
#Number of items returned with no CustomerID in June
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'June') )].Quantity.sum() #45,917

-45917

In [18]:
#Number of items returned with no CustomerID in June
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'June') )].Rev.sum() #-1,108.13

-1108.13

In [156]:
df.Return.value_counts()

0    528919
1     10040
Name: Return, dtype: int64

In [157]:
df.Rev.sum()

9792708.879999999

In [158]:
df.groupby('Country').agg({'Rev': ['sum']})

,Rev
,sum
Country,
Australia,1.369900e+05
Austria,8.698320e+03
Bahrain,5.484000e+02
Belgium,3.666296e+04
Brazil,1.143600e+03
Canada,3.115440e+03
Channel Islands,1.993629e+04
Cyprus,1.293129e+04


In [160]:
#Customer Type - Part 1 of 3

df_custtype = df.groupby('CustomerID').agg({'Rev': ['sum'], 'Quantity':['sum']})
df_custtype = df_custtype.reset_index(drop=False)
df_custtype.columns = ['CustomerID','Rev','Quantity']
df_custtype.head()


,CustomerID,Rev,Quantity
0,12346.0,0.00,0
1,12347.0,4310.00,2458
2,12348.0,1437.24,2332
3,12349.0,1457.55,630
4,12350.0,294.40,196


In [161]:
#Customer Type - Part 2 of 3

def customertype(row):
    """
    Input: Revenue and Total Quantity of Units Purchased
    Output: Flag to determing if a business (B2B) or regular (B2C) customer
    
    The purpose of this funtion is to determine if the customer ID belongs to a business or regular customer
    """
    #try:
        
    if row.Rev > 2500 or  row.Quantity > 50:
        return 1
    else:
        return 0
    #except:
        #pass
    
df_custtype['B2B'] = df_custtype.apply(customertype, axis=1)
df_custtype.head()

,CustomerID,Rev,Quantity,B2B
0,12346.0,0.00,0,0
1,12347.0,4310.00,2458,1
2,12348.0,1437.24,2332,1
3,12349.0,1457.55,630,1
4,12350.0,294.40,196,1


In [162]:
#Customer Type - Part 3 of 3

def customertypelookup(row):
    """
    Input: CustomerID
    Output: Flat to identify if B2B customer or not (1 or 0)
    
    The purpose of this funtion is to vlookup if the customer ID belongs to a business or regular customer
    """
    try:
        return int(df_custtype.loc[df_custtype['CustomerID'] == row]["B2B"])
    except:
        pass
    
df['B2B'] = df['CustomerID'].apply(customertypelookup)
df.head()

C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev,B2B
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,December,0,15.30,1.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34,1.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,December,0,22.00,1.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34,1.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34,1.0


In [164]:
df_percent = df[df.Return == 1].groupby('B2B').agg({'Rev': ['sum']})

In [165]:
df_percent.head()

,Rev
,sum
B2B,
0.0,-259763.97
1.0,-214639.36


In [167]:
df_percent.columns = ['Rev']

In [168]:
df_percent.head()

,Rev
B2B,
0.0,-259763.97
1.0,-214639.36


In [170]:
df_percent['Rev'].value_counts(normalize=True) * 100

-259763.97    50.0
-214639.36    50.0
Name: Rev, dtype: float64

In [178]:
df_ret_items = df[(df.Return == 1)&(~df.CustomerID.isna())].groupby('B2B').agg({'Quantity': ['sum']})

In [176]:
df_ret_items

,Quantity
,sum
B2B,


In [179]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return,day,month,year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,1,12,2010
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,1,12,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,1,12,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,1,12,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,1,12,2010


In [ ]:
Investigating to determine the relevance of preceding 'C' in the invoice number...

In [180]:
df_raw['FLInv'] = df_raw['InvoiceNo'].astype(str).str[0]

In [184]:
len(df_raw[(df_raw['FLInv'] != 'C') & (df_raw['Return'] ==1)].Description.unique())

139

In [186]:
df_raw[(df_raw['FLInv'] != 'C') & (df_raw['Return'] ==1)].CustomerID.unique()

array([nan])

In [191]:
df_raw[(df_raw['FLInv'] == 'C') & (df_raw['Return'] ==1)].CustomerID.isna().sum()

383

In [ ]:
#now to try with the cleaned Data Set

In [192]:
df['FLInv'] = df['InvoiceNo'].astype(str).str[0]

C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [204]:
df[(df['FLInv'] != 'C') & (df['Return'] == 1)].Rev.sum()

0.0

In [214]:
df_tmp = df[(df['FLInv'] == 'C') & (df['Return'] == 1)].groupby('Country').agg({'Rev': ['sum']})#.sort_values(by=df.columns[0])

In [216]:
df_tmp.sort_values(by=df_tmp.columns[0])

,Rev
,sum
Country,
United Kingdom,-445843.38
EIRE,-11500.84
Germany,-4761.49
Spain,-3959.91
France,-2419.35
Japan,-1996.58
USA,-1849.47
Sweden,-1662.42


In [220]:
df_tmp.value_counts(normalize=True) * 100

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [221]:
df_tmp_item = df[(df['FLInv'] == 'C') & (df['Return'] == 1)].groupby('Country').agg({'Quantity': ['sum']})#.sort_values(by=df.columns[0])

In [227]:
df[(df['FLInv'] == 'C') & (df['Return'] == 1)].Quantity.sum()

-272030

In [228]:
df[(df['FLInv'] == 'C') & (df['Return'] == 1) & (df['CustomerID'] != 15749) & (df['CustomerID'] != 12346)].Quantity.sum()

-188801

In [226]:
df_tmp_item.columns[0]

('Quantity', 'sum')

In [222]:
df_tmp_item.sort_values(by=df_tmp_item.columns[0])

,Quantity
,sum
Country,
United Kingdom,-258099
EIRE,-4800
Germany,-1798
France,-1579
USA,-1424
Spain,-1124
Japan,-795
Australia,-555


In [19]:
df_awp_stg = df[(df['CustomerID'] > 0)]

In [20]:
df_awp = df_awp_stg.groupby('StockCode').agg({'Rev': ['sum'], 'Quantity':['sum']})
df_awp = df_awp.reset_index(drop=False)
df_awp.columns = ['StockCode','Rev','Quantity']
df_awp['AWP'] = df_awp.Rev / df_awp.Quantity
df_awp.head()

,StockCode,Rev,Quantity,AWP
0,10002,699.55,823,0.850000
1,10080,114.41,291,0.393162
2,10120,40.53,193,0.210000
3,10125,930.30,1226,0.758809
4,10133,1139.41,2374,0.479954


In [21]:
#Working version
def awp_lookup(row):
    """
    Input: StockCode taken from row within dataframe
    Output: The corresponding AWP of that item
    
    The purpose of this function is to vlookup value of item from a reference dataframe (df_awp)
    """
    if row is not None:
        try:
            awp = df_awp.loc[(df_awp['StockCode'] == row.StockCode)]["AWP"]
            return awp
        except:
            return 0
    else:
        return 0

In [22]:
df_returns = df[  (df.Return == 1) ]

In [23]:
df_awp = df_awp_stg.groupby('StockCode').agg({'Rev': ['sum'], 'Quantity':['sum']})
df_awp = df_awp.reset_index(drop=False)
df_awp.columns = ['StockCode','Rev','Quantity']
df_awp['AWP'] = df_awp.Rev / df_awp.Quantity
df_awp.head()





,StockCode,Rev,Quantity,AWP
0,10002,699.55,823,0.850000
1,10080,114.41,291,0.393162
2,10120,40.53,193,0.210000
3,10125,930.30,1226,0.758809
4,10133,1139.41,2374,0.479954


In [24]:
df_returns.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,December,1,-4.65
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,December,1,-19.80
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,December,1,-6.96
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,December,1,-6.96
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,December,1,-6.96


In [25]:
df_returns = df_returns.groupby('StockCode').agg({'Rev': ['sum'], 'Quantity':['sum']})
df_returns = df_returns.reset_index(drop=False)


In [26]:
df_returns.columns = ['StockCode','Rev','Quantity']
df_returns.head()

,StockCode,Rev,Quantity
0,10002,0.00,-3
1,10133,-4.20,-92
2,10134,0.00,-19
3,10135,-1.25,-1
4,11001,-237.05,-185


In [27]:
df_awp.head()

,StockCode,Rev,Quantity,AWP
0,10002,699.55,823,0.850000
1,10080,114.41,291,0.393162
2,10120,40.53,193,0.210000
3,10125,930.30,1226,0.758809
4,10133,1139.41,2374,0.479954


In [28]:
#df_returns = df_returns.join(df_awp, lsuffix='StockCode', rsuffix='StockCode')
df_returns = pd.merge(df_returns, df_awp, how='left', left_on='StockCode', right_on='StockCode')
df_returns.head()

,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP
0,10002,0.00,-3,699.55,823.0,0.850000
1,10133,-4.20,-92,1139.41,2374.0,0.479954
2,10134,0.00,-19,NaN,NaN,NaN
3,10135,-1.25,-1,1785.44,1937.0,0.921755
4,11001,-237.05,-185,1715.95,1067.0,1.608201


In [29]:
df_returns.head()


,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP
0,10002,0.00,-3,699.55,823.0,0.850000
1,10133,-4.20,-92,1139.41,2374.0,0.479954
2,10134,0.00,-19,NaN,NaN,NaN
3,10135,-1.25,-1,1785.44,1937.0,0.921755
4,11001,-237.05,-185,1715.95,1067.0,1.608201


In [30]:
df_returns['ActRefundGBP'] = df_returns['Quantity_x']*df_returns['AWP']

In [31]:
df_returns.head()

,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP,ActRefundGBP
0,10002,0.00,-3,699.55,823.0,0.850000,-2.550000
1,10133,-4.20,-92,1139.41,2374.0,0.479954,-44.155737
2,10134,0.00,-19,NaN,NaN,NaN,NaN
3,10135,-1.25,-1,1785.44,1937.0,0.921755,-0.921755
4,11001,-237.05,-185,1715.95,1067.0,1.608201,-297.517104


In [32]:
df_returns['ActRefundGBP'].sum()

-inf

In [33]:
df_returns['ActRefundGBP'].sort_values()

1964            -inf
1473   -8.570707e+04
2219   -3.265650e+04
2152   -3.133939e+04
1053   -1.654703e+04
2217   -1.357425e+04
1052   -1.246267e+04
1430   -1.109490e+04
902    -1.046405e+04
1356   -7.812775e+03
2431   -7.786722e+03
52     -7.596446e+03
1434   -7.132950e+03
1431   -6.848110e+03
1432   -6.845878e+03
1433   -6.552000e+03
966    -5.818589e+03
190    -4.868209e+03
2196   -4.625068e+03
411    -4.603536e+03
224    -4.590601e+03
1796   -4.413274e+03
1193   -4.036395e+03
539    -4.000119e+03
412    -3.984875e+03
800    -3.950444e+03
2195   -3.809836e+03
1354   -3.780250e+03
1808   -3.759505e+03
1612   -3.729355e+03
            ...     
2353             NaN
2367             NaN
2368             NaN
2369             NaN
2370             NaN
2371             NaN
2392             NaN
2447             NaN
2456             NaN
2469             NaN
2478             NaN
2494             NaN
2497             NaN
2500             NaN
2512             NaN
2515             NaN
2517         

In [34]:
df_returns.iloc[1964]

StockCode       85047
Rev_x          -18.25
Quantity_x         -3
Rev_y             0.8
Quantity_y          0
AWP               inf
ActRefundGBP     -inf
Name: 1964, dtype: object

In [35]:
df_awp[df_awp['StockCode'] == '85047']

,StockCode,Rev,Quantity,AWP


In [36]:
df[df['StockCode'] == '85047']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev


In [37]:
df_raw[df_raw['StockCode'] == '85047']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [38]:
df_awp[df_awp['StockCode'] == '85047']

,StockCode,Rev,Quantity,AWP


In [39]:
df_returns[df_returns['StockCode'] == '85047']

,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP,ActRefundGBP


In [40]:
df_returnsB = df_returns.drop(1964)

In [41]:
df_returnsB.head()

,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP,ActRefundGBP
0,10002,0.00,-3,699.55,823.0,0.850000,-2.550000
1,10133,-4.20,-92,1139.41,2374.0,0.479954,-44.155737
2,10134,0.00,-19,NaN,NaN,NaN,NaN
3,10135,-1.25,-1,1785.44,1937.0,0.921755,-0.921755
4,11001,-237.05,-185,1715.95,1067.0,1.608201,-297.517104


In [42]:
df_returnsB.ActRefundGBP.sum()

-582089.2195770355

In [43]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [44]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,December,0,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,December,0,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34


In [48]:
df[df['Return'] ==1 ].Rev.sum()

-478724.18000000005

In [49]:
df[df['Return'] ==1 ].Rev.describe()

count     10040.000000
mean        -47.681691
std        1854.039950
min     -168469.600000
25%         -15.000000
50%          -6.240000
75%          -1.950000
max          -0.000000
Name: Rev, dtype: float64

In [51]:
df[df['Rev'] == -168469.60 ]#.Rev.describe()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,December,1,-168469.6


In [53]:
df[df['StockCode'] == '23843' ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev


In [59]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
Month                  object
Return                  int64
Rev                   float64
dtype: object

In [60]:
df[df['StockCode'] == 23843 ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,December,0,168469.6
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,December,1,-168469.6


In [54]:
df[df['InvoiceNo'] == 'C581484' ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,December,1,-168469.6


In [55]:
df[df['CustomerID'] == 16446 ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
194354,553573,22980,PANTRY SCRUBBING BRUSH,1,2011-05-18 09:52:00,1.65,16446.0,United Kingdom,May,0,1.65
194355,553573,22982,PANTRY PASTRY BRUSH,1,2011-05-18 09:52:00,1.25,16446.0,United Kingdom,May,0,1.25
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,December,0,168469.60
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,December,1,-168469.60


In [77]:
df[df['StockCode'] == 23843 ]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,December,0,168469.6
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,December,1,-168469.6


In [61]:
df_revised = df

In [67]:
df_revised.iloc[540420:540423]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev


In [69]:
df_raw_revised = df_raw

In [71]:
df_raw_revised.iloc[540421:540423]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom


In [73]:
df.iloc[540420:540423]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev


In [75]:
df_raw.iloc[540421:540423]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom


In [84]:
df[(df['InvoiceDate'].dt.year == 2010) & (df['Return'] ==1)].Rev.sum() #Dec 2010

-17607.86

In [85]:
df[(df['InvoiceDate'].dt.year == 2011) & (df['Return'] ==1) & (df['InvoiceDate'].dt.month == 1) ].Rev.sum() #Jan 2011 GBP

-91526.29999999999

In [87]:
df[(df['InvoiceDate'].dt.year == 2011) & (df['Return'] ==1) & (df['InvoiceDate'].dt.month == 1) ].Quantity.sum() #Jan 2011 Units

-88689

In [88]:
df[ (df['Return'] ==1) & (df['InvoiceDate'].dt.month != 12) ].Quantity.sum() #Jan-Nov 2011 Units

-370244

In [89]:
df[ (df['Return'] ==1) & (df['InvoiceDate'].dt.month != 12) ].Rev.sum() #Jan-Nov 2011 GBP

-286989.56

In [90]:
df[ (df['Return'] ==1) & (df['InvoiceDate'].dt.month != 12) ].Rev.describe()

count     8894.000000
mean       -32.267772
std        830.557990
min     -77183.600000
25%        -15.000000
50%         -5.950000
75%         -1.950000
max         -0.000000
Name: Rev, dtype: float64

In [91]:
df[ (df['Return'] ==1) & (df['InvoiceDate'].dt.month != 12) &  (df['Rev'] ==-77183.6)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,January,1,-77183.6


In [93]:
df[df['StockCode'] == 23166]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,January,0,77183.60
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,January,1,-77183.60
186770,552882,23166,MEDIUM CERAMIC TOP STORAGE JAR,96,2011-05-12 10:10:00,1.04,14646.0,Netherlands,May,0,99.84
187196,552953,23166,MEDIUM CERAMIC TOP STORAGE JAR,4,2011-05-12 12:11:00,1.25,16745.0,United Kingdom,May,0,5.00
187718,553005,23166,MEDIUM CERAMIC TOP STORAGE JAR,5,2011-05-12 16:29:00,1.25,14651.0,United Kingdom,May,0,6.25
187868,553009,23166,MEDIUM CERAMIC TOP STORAGE JAR,3,2011-05-12 16:52:00,1.25,13601.0,United Kingdom,May,0,3.75
188644,553052,23166,MEDIUM CERAMIC TOP STORAGE JAR,48,2011-05-13 10:14:00,1.04,15251.0,United Kingdom,May,0,49.92
189398,553149,23166,MEDIUM CERAMIC TOP STORAGE JAR,12,2011-05-13 14:16:00,1.25,14226.0,United Kingdom,May,0,15.00
189452,553152,23166,MEDIUM CERAMIC TOP STORAGE JAR,12,2011-05-13 14:55:00,1.25,13089.0,United Kingdom,May,0,15.00
189591,553160,23166,MEDIUM CERAMIC TOP STORAGE JAR,12,2011-05-13 15:26:00,1.25,14194.0,United Kingdom,May,0,15.00


In [95]:
df.sort_values(by='Rev')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,December,1,-168469.60
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,January,1,-77183.60
160145,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,2011-04-18 13:08:00,2.10,15749.0,United Kingdom,April,1,-6539.40
160143,C550456,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1930,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,April,1,-4921.50
160141,C550456,48185,DOORMAT FAIRY CAKE,-670,2011-04-18 13:08:00,6.75,15749.0,United Kingdom,April,1,-4522.50
349750,C567527,23113,PANTRY CHOPPING BOARD,-756,2011-09-21 09:16:00,5.06,17450.0,United Kingdom,September,1,-3825.36
160144,C550456,21175,GIN + TONIC DIET METAL SIGN,-2000,2011-04-18 13:08:00,1.85,15749.0,United Kingdom,April,1,-3700.00
160142,C550456,47566B,TEA TIME PARTY BUNTING,-1300,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,April,1,-3315.00
320580,C565044,22191,IVORY DINER WALL CLOCK,-318,2011-08-31 17:02:00,7.65,12931.0,United Kingdom,August,1,-2432.70
390547,C570556,22273,FELTCRAFT DOLL MOLLY,-720,2011-10-11 11:10:00,2.55,16029.0,United Kingdom,October,1,-1836.00


In [99]:
#VERSION 1 - Finding Customers with sale and return on the same day - Part 1 of 2
df_buy_return_SD = df.groupby(['CustomerID','StockCode','Return'])['Quantity'].unique()

In [100]:
#VERSION 1 - Finding Customers with sale and return on the same day - Part 2 of 2
df_buy_return_SD.loc[df_buy_return_SD.apply(lambda x:len(x)>1)]

CustomerID  StockCode  Return
12347.0     21064      0                  [6, 24]
            21731      0                 [12, 24]
            21791      0                 [12, 24]
            22195      0                 [12, 24]
            22196      0                 [12, 24]
            22371      0               [4, 8, 12]
            22374      0                   [4, 8]
            22375      0                   [4, 8]
            22423      0                   [3, 6]
            22699      0                  [6, 18]
            22725      0                   [4, 8]
            22726      0                   [4, 8]
            22727      0               [4, 8, 12]
            22728      0                   [4, 8]
            23084      0             [12, 48, 24]
            23146      0                   [4, 8]
            85116      0                  [12, 6]
            84558A     0         [24, 12, 18, 36]
12348.0     21213      0                [24, 120]
            21977   

In [101]:
#VERSION 2 - Finding Customers with sale and return on the same day - Part 1 of 2
df_buy_return_SD = df.groupby(['CustomerID','StockCode','Month','Return'])['Quantity'].unique()

In [102]:
#VERSION 2 - Finding Customers with sale and return on the same day - Part 2 of 2
df_buy_return_SD.loc[df_buy_return_SD.apply(lambda x:len(x)>1)]

CustomerID  StockCode  Month      Return
12347.0     21064      December   0                [6, 24]
            21731      December   0               [12, 24]
12348.0     21213      December   0              [24, 120]
            21977      December   0              [24, 120]
            22952      December   0              [24, 120]
            84991      December   0              [24, 120]
12359.0     22423      October    0              [4, 6, 3]
12362.0     22352      October    0                [12, 6]
12391.0     20985      November   0                 [1, 2]
12393.0     23389      September  0                 [4, 8]
12395.0     22661      November   0               [20, 10]
12406.0     22383      June       0              [100, 25]
12407.0     84536B     July       0              [16, 272]
12409.0     23298      September  0                [1, 50]
12415.0     21503      July       0               [200, 1]
            22722      October    0             [240, 120]
            230

In [112]:
df_raw['Return'] = df_raw.apply(transtype, axis=1)
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0


In [113]:
#VERSION 3 - Finding Customers with sale and return on the same day - Part 1 of 2
df_buy_return_SDr = df_raw.groupby(['CustomerID','StockCode','Quantity'])['Return'].unique()

In [114]:
#VERSION 3 - Finding Customers with sale and return on the same day - Part 2 of 2
df_buy_return_SDr.loc[df_buy_return_SDr.apply(lambda x:len(x)>1)]

Series([], Name: Return, dtype: object)

In [115]:
df_buy_return_SDr.head()

CustomerID  StockCode  Quantity
12346.0     23166      -74215      [1]
                        74215      [0]
12347.0     16008       24         [0]
            17021       36         [0]
            20665       6          [0]
Name: Return, dtype: object

In [116]:
df_raw[df_raw.InvoiceNo == 'C550456']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return
160141,C550456,48185,DOORMAT FAIRY CAKE,-670,2011-04-18 13:08:00,6.75,15749.0,United Kingdom,1
160142,C550456,47566B,TEA TIME PARTY BUNTING,-1300,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,1
160143,C550456,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1930,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,1
160144,C550456,21175,GIN + TONIC DIET METAL SIGN,-2000,2011-04-18 13:08:00,1.85,15749.0,United Kingdom,1
160145,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,2011-04-18 13:08:00,2.10,15749.0,United Kingdom,1


In [117]:
df_raw[(df_raw.CustomerID == 15749.0) ]#& ()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return
52709,540815,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1930,2011-01-11 12:55:00,2.55,15749.0,United Kingdom,0
52710,540815,21175,GIN + TONIC DIET METAL SIGN,2000,2011-01-11 12:55:00,1.85,15749.0,United Kingdom,0
52711,540815,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-01-11 12:55:00,2.10,15749.0,United Kingdom,0
52771,540818,47556B,TEA TIME TEA TOWELS,1300,2011-01-11 12:57:00,2.55,15749.0,United Kingdom,0
52772,540818,48185,DOORMAT FAIRY CAKE,670,2011-01-11 12:57:00,6.75,15749.0,United Kingdom,0
160141,C550456,48185,DOORMAT FAIRY CAKE,-670,2011-04-18 13:08:00,6.75,15749.0,United Kingdom,1
160142,C550456,47566B,TEA TIME PARTY BUNTING,-1300,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,1
160143,C550456,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1930,2011-04-18 13:08:00,2.55,15749.0,United Kingdom,1
160144,C550456,21175,GIN + TONIC DIET METAL SIGN,-2000,2011-04-18 13:08:00,1.85,15749.0,United Kingdom,1
160145,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,2011-04-18 13:08:00,2.10,15749.0,United Kingdom,1


In [119]:
df_raw[df_raw.StockCode == 23166].sort_values(by='')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,0
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,1
186770,552882,23166,MEDIUM CERAMIC TOP STORAGE JAR,96,2011-05-12 10:10:00,1.04,14646.0,Netherlands,0
187196,552953,23166,MEDIUM CERAMIC TOP STORAGE JAR,4,2011-05-12 12:11:00,1.25,16745.0,United Kingdom,0
187718,553005,23166,MEDIUM CERAMIC TOP STORAGE JAR,5,2011-05-12 16:29:00,1.25,14651.0,United Kingdom,0
187868,553009,23166,MEDIUM CERAMIC TOP STORAGE JAR,3,2011-05-12 16:52:00,1.25,13601.0,United Kingdom,0
188644,553052,23166,MEDIUM CERAMIC TOP STORAGE JAR,48,2011-05-13 10:14:00,1.04,15251.0,United Kingdom,0
189398,553149,23166,MEDIUM CERAMIC TOP STORAGE JAR,12,2011-05-13 14:16:00,1.25,14226.0,United Kingdom,0
189452,553152,23166,MEDIUM CERAMIC TOP STORAGE JAR,12,2011-05-13 14:55:00,1.25,13089.0,United Kingdom,0
189591,553160,23166,MEDIUM CERAMIC TOP STORAGE JAR,12,2011-05-13 15:26:00,1.25,14194.0,United Kingdom,0


In [ ]:
df['year'] = df['date'].year
df['month'] = df['date'].month

In [127]:
df_raw['day'] = df_raw['InvoiceDate'].dt.day
df_raw['month'] = df_raw['InvoiceDate'].dt.month
df_raw['year'] = df_raw['InvoiceDate'].dt.year

In [128]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return,day,month,year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,1,12,2010
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,1,12,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,1,12,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,1,12,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,1,12,2010


In [135]:
#VERSION 5 - Finding Customers with sale and return on the same day - Part 1 of 2
df_buy_return_SDr = df_raw.groupby(['CustomerID','StockCode','day','month','year'])[('Quantity')].unique()

In [136]:
#VERSION 5 - Finding Customers with sale and return on the same day - Part 2 of 2
df_buy_return_SDr.loc[df_buy_return_SDr.apply(lambda x:len(x)>1)]

CustomerID  StockCode  day  month  year
12346.0     23166      18   1      2011    [74215, -74215]
12348.0     21213      16   12     2010          [24, 120]
            21977      16   12     2010          [24, 120]
            22952      16   12     2010          [24, 120]
            84991      16   12     2010          [24, 120]
12352.0     21914      22   3      2011          [-12, 12]
            22413      22   3      2011            [-6, 6]
            22645      22   3      2011          [-12, 12]
            22701      22   3      2011            [-6, 6]
            22784      22   3      2011            [-3, 3]
            37448      22   3      2011          [-12, 12]
            84050      22   3      2011          [-12, 12]
            M          1    3      2011            [-1, 1]
12359.0     22423      13   10     2011          [4, 6, 3]
12365.0     M          21   2      2011            [-1, 1]
12391.0     20985      18   11     2011             [1, 2]
12393.0     2338

In [140]:
df_raw.sort_values(by='Quantity').head(80)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return,day,month,year
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,1,9,12,2011
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,1,18,1,2011
225529,556690,23005,printing smudges/thrown away,-9600,2011-06-14 10:37:00,0.00,NaN,United Kingdom,1,14,6,2011
225530,556691,23005,printing smudges/thrown away,-9600,2011-06-14 10:37:00,0.00,NaN,United Kingdom,1,14,6,2011
4287,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom,1,2,12,2010
225528,556687,23003,Printing smudges/thrown away,-9058,2011-06-14 10:36:00,0.00,NaN,United Kingdom,1,14,6,2011
115818,546152,72140F,throw away,-5368,2011-03-09 17:25:00,0.00,NaN,United Kingdom,1,9,3,2011
431381,573596,79323W,"Unsaleable, destroyed.",-4830,2011-10-31 15:17:00,0.00,NaN,United Kingdom,1,31,10,2011
341601,566768,16045,NaN,-3667,2011-09-14 17:53:00,0.00,NaN,United Kingdom,1,14,9,2011
323458,565304,16259,NaN,-3167,2011-09-02 12:18:00,0.00,NaN,United Kingdom,1,2,9,2011


In [148]:
df_raw[df_raw.CustomerID.isnull()].sort_values(by='Quantity').head(300)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return,day,month,year
225530,556691,23005,printing smudges/thrown away,-9600,2011-06-14 10:37:00,0.0,NaN,United Kingdom,1,14,6,2011
225529,556690,23005,printing smudges/thrown away,-9600,2011-06-14 10:37:00,0.0,NaN,United Kingdom,1,14,6,2011
225528,556687,23003,Printing smudges/thrown away,-9058,2011-06-14 10:36:00,0.0,NaN,United Kingdom,1,14,6,2011
115818,546152,72140F,throw away,-5368,2011-03-09 17:25:00,0.0,NaN,United Kingdom,1,9,3,2011
431381,573596,79323W,"Unsaleable, destroyed.",-4830,2011-10-31 15:17:00,0.0,NaN,United Kingdom,1,31,10,2011
341601,566768,16045,NaN,-3667,2011-09-14 17:53:00,0.0,NaN,United Kingdom,1,14,9,2011
323458,565304,16259,NaN,-3167,2011-09-02 12:18:00,0.0,NaN,United Kingdom,1,2,9,2011
263884,560039,20713,wrongly marked. 23343 in box,-3100,2011-07-14 14:27:00,0.0,NaN,United Kingdom,1,14,7,2011
113580,545990,84598,check,-3000,2011-03-08 13:07:00,0.0,NaN,United Kingdom,1,8,3,2011
375429,569466,23270,incorrect stock entry.,-2880,2011-10-04 11:42:00,0.0,NaN,United Kingdom,1,4,10,2011


In [144]:
df_raw[df_raw.CustomerID.isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return,day,month,year
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom,0,1,12,2010
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom,0,1,12,2010
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom,0,1,12,2010
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom,0,1,12,2010
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom,0,1,12,2010
1447,536544,21790,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,NaN,United Kingdom,0,1,12,2010
1448,536544,21791,VINTAGE HEADS AND TAILS CARD GAME,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom,0,1,12,2010
1449,536544,21801,CHRISTMAS TREE DECORATION WITH BELL,10,2010-12-01 14:32:00,0.43,NaN,United Kingdom,0,1,12,2010
1450,536544,21802,CHRISTMAS TREE HEART DECORATION,9,2010-12-01 14:32:00,0.43,NaN,United Kingdom,0,1,12,2010
1451,536544,21803,CHRISTMAS TREE STAR DECORATION,11,2010-12-01 14:32:00,0.43,NaN,United Kingdom,0,1,12,2010


In [ ]:
.isnull()

In [150]:
df_raw[df_raw['CustomerID'] == 12607]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return,day,month,year
389350,570467,22551,PLASTERS IN TIN SPACEBOY,12,2011-10-10 16:06:00,1.65,12607.0,USA,0,10,10,2011
389351,570467,21915,RED HARMONICA IN BOX,12,2011-10-10 16:06:00,1.25,12607.0,USA,0,10,10,2011
389352,570467,22619,SET OF 6 SOLDIER SKITTLES,4,2011-10-10 16:06:00,3.75,12607.0,USA,0,10,10,2011
389353,570467,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-10-10 16:06:00,4.95,12607.0,USA,0,10,10,2011
389354,570467,21524,DOORMAT SPOTTY HOME SWEET HOME,2,2011-10-10 16:06:00,8.25,12607.0,USA,0,10,10,2011
389355,570467,21668,RED STRIPE CERAMIC DRAWER KNOB,12,2011-10-10 16:06:00,1.45,12607.0,USA,0,10,10,2011
389356,570467,21671,RED SPOT CERAMIC DRAWER KNOB,12,2011-10-10 16:06:00,1.45,12607.0,USA,0,10,10,2011
389357,570467,21669,BLUE STRIPE CERAMIC DRAWER KNOB,12,2011-10-10 16:06:00,1.45,12607.0,USA,0,10,10,2011
389358,570467,22090,PAPER BUNTING RETROSPOT,6,2011-10-10 16:06:00,2.95,12607.0,USA,0,10,10,2011
389359,570467,22668,PINK BABY BUNTING,5,2011-10-10 16:06:00,2.95,12607.0,USA,0,10,10,2011
